<a href="https://colab.research.google.com/github/kmk4444/Miuul_deep_learning/blob/main/end_to_end_structured_data_modeling_with_ann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# End to End Structured Data Modeling with ANN

In [1]:
import keras
import tensorflow as tf
print("Keras Current Version:", keras.__version__, "Tensorflow Current Version:", tf.__version__)

Keras Current Version: 3.8.0 Tensorflow Current Version: 2.18.0


In [ ]:
# !pip uninstall tf-keras

In [ ]:
# !pip install tensorflow==2.16.1

# Data Preparation

## Imports and Settings

In [2]:
import numpy as np
import pandas as pd
from joblib import dump, load
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization

random.seed(46)
np.random.seed(46)
tf.random.set_seed(46)

# Ortam ayarları ve veri yükleme
data_path = '/content/diabetes.csv'

## Normalizing Inputs and Save the Scaler

In [3]:
# Define a function to preprocess data from a given CSV file
def preprocess_data(filepath):

  # Read the CSV file into a DataFrame using Pandas
  data = pd.read_csv(filepath)

  # Create an instance of StandardScaler to standardize features
  scaler = StandardScaler()

  # Scale the features (all columns except 'Outcome') and store them in X
  # 'fit_transform' calculates mean and std, then scales the data
  X = scaler.fit_transform(data.drop('Outcome', axis=1))

  # Extract the 'Outcome' column as labels and convert it to a NumPy array
  y = data['Outcome'].values

  # Save the trained scaler object to a file for future use
  dump(scaler, 'scaler.joblib')

  # Return the scaled features (X) and labels (y)
  return X, y


In [4]:
X, y = preprocess_data(data_path)

## Train Validation and Tensorflow Dataset

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Define a function to prepare TensorFlow datasets for training and validation
def prepare_datasets(X_train, X_val, y_train, y_val, batch_size=None):

  # If no batch size is provided, use the entire training set as a single batch
  if batch_size is None:
    batch_size = len(X_train)

  # Create a TensorFlow dataset from training features and labels
  train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

  # Shuffle the training dataset and divide it into batches
  # buffer_size determines how much data to shuffle, here it's set to the size of the training set
  train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size)

  # Create a TensorFlow dataset from validation features and labels
  val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))

  # Divide the validation dataset into batches without shuffling
  val_dataset = val_dataset.batch(batch_size)

  # Return the prepared training and validation datasets
  return train_dataset, val_dataset


In [9]:
train_ds, val_ds = prepare_datasets(X_train, X_val, y_train, y_val)

In [10]:
train_ds

<_BatchDataset element_spec=(TensorSpec(shape=(None, 8), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

# Modeling


## Build the Model

In [11]:
# Define a Sequential model using the Keras API
model = Sequential([

    # Input layer: Specifies the shape of the input data
    # shape=(train_ds.element_spec[0].shape[1],) gets the number of features from the training dataset
    Input(shape=(train_ds.element_spec[0].shape[1],)),

    # First Dense layer with 50 neurons and ReLU activation function
    # kernel_regularizer=l2(0.001) applies L2 regularization to prevent overfitting
    Dense(50, activation='relu', kernel_regularizer=l2(0.001)),

    # Batch Normalization: Normalizes the outputs of the previous layer for faster and more stable training
    BatchNormalization(),

    # Dropout layer: Randomly sets 50% of inputs to zero during training to prevent overfitting
    Dropout(0.5),

    # Output layer: A single neuron with sigmoid activation function for binary classification
    Dense(1, activation='sigmoid')
])


In [ ]:
# Create an SGD optimizer with a learning rate of 0.01 and no momentum
# SGD (Stochastic Gradient Descent) is an optimization algorithm to update model weights
optimizer = SGD(learning_rate=0.01, momentum=0.0)

# Compile the model with the following configurations:
model.compile(
    # Use the defined SGD optimizer to adjust model weights during training
    optimizer=optimizer,

    # Specify the loss function as binary_crossentropy for binary classification tasks
    # This function measures the difference between predicted and actual labels
    loss="binary_crossentropy",

    # Define metrics to monitor during training and evaluation:
    # "accuracy" to measure the percentage of correct predictions
    # "AUC" (Area Under the Curve) to evaluate the model's ability to distinguish between classes
    metrics=["accuracy", "AUC"]
)
